## Import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import warnings
warnings.filterwarnings("ignore")

from datetime import date, timedelta
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
import os
import sys


## Downloading Data

In [ ]:
!wget --header="Host: storage.googleapis.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: en-US,en;q=0.9" --header="Referer: https://www.kaggle.com/" "https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/7391/44328/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1595846545&Signature=fOmnQiBAwPnPPle4jgShB4NFMjbmoiOoOlFQRYjPKCX7GKNFuTWdK0TLICd6qEaosYMvkam3gkTLL1DzSO1fI94bUqpM%2FnI%2FLZq8HWDJYHjeiP06zCajKO4BA3UIytFW3dy1IKbQWfTvY%2Bme7rtMOTJOAr3HpSkXiELNXNepz8G22cVCaZY1P6t2KmPhCbgIwMvuko30hVxa6W6Gx7426qT5jDlvLhdoErdMbgf9SyAkK1HG1aZc0IqAGXvNUAkm7i3qDHEh6OKVn6HijqDyYEbuzRAU3r7CA1Pyme%2Bn%2Bf1xLlIo85LaeaSQdQgFnKy9Q9ZqCKjZhqBai%2FBQsQrJrQ%3D%3D&response-content-disposition=attachment%3B+filename%3Dfavorita-grocery-sales-forecasting.zip" -c -O 'favorita-grocery-sales-forecasting.zip'
import os
    
if os.path.exists('favorita-grocery-sales-forecasting.zip'):
    !unzip 'favorita-grocery-sales-forecasting.zip'
    #print("File unzipped Successfully")
else:
    sys.exit("'favorita-grocery-sales-forecasting.zip' File Not Present to unzip")  

#installing 7zip for extracting .7z files
!apt-get install p7zip-full


for file in os.listdir():
    if file[-3:]=='.7z':
        if os.path.exists(file[:-3]):
            print("="*50)
            print("'{}'Extracted File is Already Present".format(file[:-3]))
                
        elif file=='train.csv.7z':
            !p7zip -d 'train.csv.7z'
            print("="*50)
            print("'{}' File Extracted Successfully".format(file))

        elif file=='stores.csv.7z':
            !p7zip -d 'stores.csv.7z'
            print("="*50)
            print("'{}' File Extracted Successfully".format(file))

        elif file=='items.csv.7z':
            !p7zip -d 'items.csv.7z'
            print("="*50)
            print("'{}' File Extracted Successfully".format(file))

        elif file=='test.csv.7z':
            !p7zip -d 'test.csv.7z'
            print("="*50)
            print("'{}' File Extracted Successfully".format(file))

        print("="*50)


--2020-07-25 16:16:17--  https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/7391/44328/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1595846545&Signature=fOmnQiBAwPnPPle4jgShB4NFMjbmoiOoOlFQRYjPKCX7GKNFuTWdK0TLICd6qEaosYMvkam3gkTLL1DzSO1fI94bUqpM%2FnI%2FLZq8HWDJYHjeiP06zCajKO4BA3UIytFW3dy1IKbQWfTvY%2Bme7rtMOTJOAr3HpSkXiELNXNepz8G22cVCaZY1P6t2KmPhCbgIwMvuko30hVxa6W6Gx7426qT5jDlvLhdoErdMbgf9SyAkK1HG1aZc0IqAGXvNUAkm7i3qDHEh6OKVn6HijqDyYEbuzRAU3r7CA1Pyme%2Bn%2Bf1xLlIo85LaeaSQdQgFnKy9Q9ZqCKjZhqBai%2FBQsQrJrQ%3D%3D&response-content-disposition=attachment%3B+filename%3Dfavorita-grocery-sales-forecasting.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.203.128, 74.125.204.128, 64.233.188.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.203.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 480014675 (458M) [application/zip]
Saving to: ‘favorita-grocery-sales-

## Create Features

### Preparing Data

####Reduce memory usage

In [ ]:
# Reference: https://www.kaggle.com/gemartin/load-data-reduce-memory-usage
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    import numpy as np
    
    #start_mem = df.memory_usage().sum() / 1024**2
    #print('Memory usage of Dataframe is {:.3f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object and col_type.name != 'category' and 'datetime' not in col_type.name:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        elif 'datetime' not in col_type.name:
            df[col] = df[col].astype('category')

    #end_mem = df.memory_usage().sum() / 1024**2
    #print('Memory usage after optimization is: {:.3f} MB'.format(end_mem))
    #print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

#### Function for Reading and Processing Data

In [ ]:
def read_process():
    
    import pandas as pd
    import numpy as np

    
    """### Test Data"""

    #Reading test.csv
    df_test = pd.read_csv("test.csv", dtype={'onpromotion': int},parse_dates=["date"] )

    """###Items Data"""

    #Reading items.csv
    items = pd.read_csv("items.csv",)


    #Setting item_nbr as the index of the dataframe
    items = items.set_index("item_nbr") 


    """#### Label Encoding (item family)"""

    #initializing label encoder
    encoder = LabelEncoder()
    #transforming item family column
    items['family'] = encoder.fit_transform(items['family'].values)

    """### Stores Data"""

    #Reading store.csv
    stores = pd.read_csv("stores.csv")

    # Setting store_nbr as the index of the dataframe
    stores = stores.set_index("store_nbr")

    """#### Label Encoding on store state, city and type"""

    #initializing label encoder
    encoder = LabelEncoder()
    #transforming state column
    stores['state'] = encoder.fit_transform(stores['state'].values)
    #transforming city column
    stores['city'] = encoder.fit_transform(stores['city'].values)
    #transforming type column
    stores['type'] = encoder.fit_transform(stores['type'].values)


    """### Train Data"""

    #Skipping rows from 1 to 101688780‬ i.e. reading 2017 data only.
    df_train = pd.read_csv('train.csv', dtype={'onpromotion': int}, parse_dates=["date"],converters={'unit_sales': lambda u: np.log1p(
        float(u)) if float(u) > 0 else 0}, skiprows=range(1,101688780))

    #Dropping Id Column
    df_train=df_train.drop('id',axis=1)

    
    return df_train,df_test,stores,items


#### Data based on item_nbr.


In [ ]:
def data_itemnbr(promo_2017,sales_2017):
    
    ''' Prepares Data based on  item_nbr. '''

    ##### Promotion Data

    #Grouping promotions by item no. over time and filtering only that columns that are in promo_2017 Dataframe.
    promo_item_2017 = promo_2017.groupby('item_nbr')[promo_2017.columns].sum()
    #promo_item_2017.head()

    """##### Sales Data"""

    #Grouping sales by item no. through time and keeping only that columns that are present in sales dataframe
    sales_item_2017 = sales_2017.groupby('item_nbr')[sales_2017.columns].sum()
    #sales_item_2017.head()
    
    return promo_item_2017,sales_item_2017


#### Data based on store_nbr and item_nbr pairs

In [ ]:
def data_storenbr_itemnbr(df_train,df_test):
    ''' Prepares Data based on store_nbr and item_nbr pairs. '''

    ##### Promotion Data

    #Setting store_nbr, item_nbr and date as the indices.
    promo_2017_train = df_train.set_index(["store_nbr", "item_nbr", "date"])

    #Using promotions column only and unstacking the dates
    promo_2017_train= promo_2017_train[["onpromotion"]].unstack(level=-1)

    #Filling the missing values with 0.
    promo_2017_train = promo_2017_train.fillna(0)

    #promo_2017_train.head()

    # Deleting 1st row
    promo_2017_train.columns = promo_2017_train.columns.get_level_values(1)
    promo_2017_train.head()

    #Setting store_nbr, item_nbr and date as the indices of the dataframe
    promo_2017_test = df_test.set_index(['store_nbr', 'item_nbr', 'date'])

    #Using promotions column only and unstacking the dates
    promo_2017_test = promo_2017_test[["onpromotion"]].unstack(level=-1).fillna(0)

    #Filling the missing information with 0.
    promo_2017_test = promo_2017_test.fillna(0)

    # Deleting 1st row
    promo_2017_test.columns = promo_2017_test.columns.get_level_values(1)
    #promo_2017_test.head()

    #Removing Items from test dataset that are not in train dataset 
    #Adding Items that are not in the test dataset and filling values with 0

    #Re-indexing with train dataframe index. By default values in the new index that do not have corresponding records in the dataframe are assigned NaN
    promo_2017_test = promo_2017_test.reindex(promo_2017_train.index)

    #Filling the missing information with 0.
    promo_2017_test = promo_2017_test.fillna(0)

    #promo_2017_test.head()

    #Concatenating train and test datasets
    promo_2017 = pd.concat([promo_2017_train, promo_2017_test], axis=1)
    #promo_2017.head()

    # Deleting unneccesary variables
    del promo_2017_test, promo_2017_train , df_test 

    """##### Sales Data"""

    #Setting store_nbr, item_nbr and date as the indices.
    sales_2017 = df_train.set_index(["store_nbr", "item_nbr", "date"])

    #Using unit_sales column only and unstacking the dates
    sales_2017 = sales_2017[["unit_sales"]].unstack(level=-1)

    #Filling the missing values with 0's.
    sales_2017 = sales_2017.fillna(0)
    # Deleting 1st row
    sales_2017.columns = sales_2017.columns.get_level_values(1)

    #sales_2017.head()
    del df_train
    
    return sales_2017,promo_2017

#### Data based on item_class and store_nbr pairs.

In [ ]:
def data_itemclass_storenbr(items ,stores,sales_2017 ,promo_2017):
    ''' Prepares Data based on item_class and store_nbr pairs. '''
    
    #Removing items that are not present in year 2017.

    #Re-indexing by using index of sales_2017 dataframe.
    items = items.reindex(sales_2017.index.get_level_values(1))

    #Removing stores that are not present in year 2017.

    #Re-indexing by using index of sales_2017 dataframe.
    stores = stores.reindex(sales_2017.index.get_level_values(0))

    """##### Promotion Data"""

    #Fetching promotion data with new sequential index
    store_class_promo_2017 = promo_2017.reset_index()

    #Adding class column
    store_class_promo_2017['class'] = items['class'].values

    #Grouping promotions by item class and store no. and filtering on;y that columns that are in promo_2017 Dataframe
    store_class_promo_2017 = store_class_promo_2017.groupby(['class', 'store_nbr'])[promo_2017.columns].sum()
    #store_class_promo_2017.head()

    """##### Sales Data"""

    #Fetching sales data with new sequential index
    store_class_sales_2017 = sales_2017.reset_index()

    #Adding class column
    store_class_sales_2017['class'] = items['class'].values 

    # Storing Item class and store_nbr pairs used for indexing later
    store_class_index = store_class_sales_2017[['class', 'store_nbr']]

    #Grouping sales by item class and store no. and keeping only that columns present in sales dataframe
    store_class_sales_2017 = store_class_sales_2017.groupby(['class', 'store_nbr'])[sales_2017.columns].sum()
    
    return items,stores,store_class_promo_2017,store_class_index,store_class_sales_2017


### Feature Engineering

#### Function for Filtering Data Between dates


In [ ]:
def get_timespan(df, Date, minus, periods, freq='D'):
    '''Selects ony that dataframe columns which correspond to 
    the "periods" days after the (Date-minus) day .'''

    
    if minus!=0:
        return df[pd.date_range(Date - timedelta(days=minus), periods=periods, freq=freq)]
    else:
        return df[pd.date_range(Date , periods=periods, freq=freq)]

#### Function for Creating Promotional Features

In [ ]:
def promo_features(promotions,Date,X):
    '''Creates features related to promotion'''

    
    # Sum of Promotions with past data at different day intervals.
    for n_days in [14,60,140]:

        # Filtering promotions of items from (date-n_days) to (date).
        filtered_promo = get_timespan(promotions, Date, n_days, n_days)

        # Sum of Promotions for each item sold over date.
        X['%sdays_promo_sum (past)'%n_days]=filtered_promo.sum(axis=1).values

    # Sum of Promotions with future data at different day intervals.
    future_date = Date + timedelta(days=1)  #Shifting date ahead 
    for n_days in [3,7,14]:

        # Filtering promotions of items from (date) to (date+n_days)
        filtered_promo = get_timespan(promotions, future_date, 0, n_days)

        # Sum of Promotions for each item sold over date.
        X['%sdays_promo_sum (future)'%n_days]=filtered_promo.sum(axis=1).values


    # Promotion feature (i.e. if there is a promotion or not) for 16 days in past and future.
    for n_day in range(-16, 16):

        if n_day<0:
            flag='past'
        elif n_day > 0:
            flag='future'
        else:
            flag='present'

        #Promotion feature for each item sold on n_day.
        X["promo_day{} ({})".format(abs(n_day),flag)] = promotions[Date + timedelta(days=n_day)].values.astype(np.uint8)

    return X


#### Function for Creating Sales Features

In [ ]:

def sales_features(sales,Date,X,past_week=False):
    '''Creates features related to Sales'''
    
    name=''
    if past_week==True:
        name=" (past_week)"
        Date = Date - timedelta(days=7) # past_week_date is date - 7 days

    for n_days in [3, 7, 14, 30, 60, 140]:

        # Filtering sales of items n_days before t2017 to t2017
        filtered_sales = get_timespan(sales, Date, n_days, n_days)

        # mean of sales of each item over date
        X['%sdays_sale_mean' % n_days + name] = filtered_sales.mean(axis=1).values

        # exponentially weighted sum_of_sales of each item over date
        exp_weights = np.power(0.9, np.arange(n_days)[::-1])
        X['%sdays_weighted_sale' % n_days + name] = (filtered_sales * exp_weights ).sum(axis=1).values

        # mean of difference in sales of each item over date
        X['%sdays_sale_diff_mean' % n_days + name] = filtered_sales.diff(axis=1).mean(axis=1).values

        # median of sales of each item over date
        X['%sdays_sale_median' % n_days + name] = filtered_sales.median(axis=1).values

        # min. of sales of each item over date
        X['%sdays_min_sale' % n_days + name] = filtered_sales.min(axis=1).values

        # max. of sales of each item over date
        X['%sdays_max_sale' % n_days + name] = filtered_sales.max(axis=1).values

        # std. of sales of each item over date
        X['%sdays_sale_std' % n_days + name] = filtered_sales.std(axis=1).values


    # Sales on the nth day in past
    for n_day in range(1, 16):
        X['sales(past_day_%s)' % n_day] = get_timespan(sales, Date, n_day, 1).values.ravel()


    
    for n_day in range(7):
        # mean of sales every same day of week during 4 weeks before today
        X['dow%s_mean_sales(4weeks)' % n_day ] = get_timespan(sales, Date, 28-n_day, 4, freq='7D').mean(axis=1).values
        # mean of sales every same day of week during  20 weeks before today
        X['dow%s_mean_sales(20weeks)' % n_day] = get_timespan(sales, Date, 140-n_day, 20, freq='7D').mean(axis=1).values
    
    
    return X


#### Function for Creating Sales Features Depending on Promotion

In [ ]:
def sales_promo_features(sales,promotions,Date,X):
    '''Creates features related to Sales and promotion'''

    for n_days in [3, 7, 14, 30, 60, 140]:

        # Filtering sales of items from (date-n_days) to (date)
        filtered_sales = get_timespan(sales, Date, n_days, n_days)

        # Filtering promotion on items from (date-n_days) to (date)
        filtered_promo = get_timespan(promotions, Date, n_days, n_days)

        # mean_of_sales of each item sold on promotion over date
        sales_with_promo = filtered_sales * filtered_promo.replace(0, np.nan)           #replacing 0's with nan so that these values get ignored while calculating mean.
        X['%sdays_sale_mean(promo)' % n_days] =sales_with_promo.mean(axis=1).values     #pandas DataFrame.mean ignore nan values

        # exponentially weighted sum_of_sales of each item sold on promotion over date
        exp_weights = np.power(0.9, np.arange(n_days-1,-1,-1))                            
        X['%sdays_weighted_sale(promo)' % n_days] = (sales_with_promo * exp_weights).sum(axis=1).values   #Giving more weightage to recent dated sales and decreasing weight with date.
        
        # mean_of_sales of each item sold without promotion over date
        sales_without_promo = filtered_sales * (1 - filtered_promo).replace(0, np.nan)       #replacing 0's with nan so that these values get ignored while calculating mean.
        X['%sdays_sale_mean(no_promo)' % n_days] = sales_without_promo.mean(axis=1).values   #pandas DataFrame.mean ignore nan values

        # exponentially weighted sum_of_sales of each item sold without promotion over date
        X['%sdays_weighted_sale(no_promo)' % n_days] = ( sales_without_promo * exp_weights).sum(axis=1).values 

    return X

#### Function for Creating Count Features

In [ ]:
def count_features(data,Date,X,name):
    '''Creates Count features related to Sales and promotion'''

    
    # Number of days a sale / promotion took place in the time window, and days since first / last sale / promotion
    for n_days in [7, 14, 30, 60, 140]:

        # Filtering data from (date-n_days) to (date)
        filtered_data = get_timespan(data, Date, n_days, n_days)

        # Number of days a sale/promotion was made/present (i.e. not equal to 0) for each item
        X['num_days_having%s(last_%sdays)' % (name,n_days)] = (filtered_data > 0).sum(axis=1).values

        # Number of days since last sale/promotion (in n_days) for each item
        X['num_days_since_Last%s(last_%sdays)'  % (name,n_days)] = n_days - ((filtered_data > 0) * np.arange(n_days)).max(axis=1).values

        # Number of days since first sale/promotion (in n_days) for each item
        X['num_days_since_First%s(last_%sdays)' % (name,n_days)] = ((filtered_data > 0) * np.arange(n_days, 0, -1)).max(axis=1).values

    if name=='Promo':
        # Number of promotions in the next 15 days, time before first and last promotion in the same time window
        Date = Date + timedelta(days=16)
        filtered_promo = get_timespan(data,Date, 15, 15)
        X['num_days_having%s(after_%sdays)' % (name,n_days)] = (filtered_promo > 0).sum(axis=1).values
        X['num_days_since_Last%s(after_%sdays)'  % (name,n_days)] = 15 - ((filtered_promo > 0) * np.arange(15)).max(axis=1).values
        X['num_days_since_First%s(after_%sdays)' % (name,n_days)] = ((filtered_promo > 0) * np.arange(15, 0, -1)).max(axis=1).values

    return X


#### Function for Creating All the Custom Features

In [ ]:
def custom_features(sales, promotions, Date, name_prefix=None):
    '''Creates all the custom features'''

    #Creating empty dictionary for adding features
    X={}
    # Features only dependent on promotions
    X = promo_features(promotions,Date,X)
    X = count_features(promotions,Date,X,name='Promo')

    # Features only dependent on unit_sales
    X = sales_features(sales,Date,X)
    X = count_features(sales,Date,X,name='Sales')
    # For Past_week
    X = sales_features(sales,Date,X,past_week=True)

    # Features dependent on both promotions and unit_sales
    X = sales_promo_features(sales,promotions,Date,X)

    # Creating Dataframe from dictionary having keys as column names and values as column values
    X = pd.DataFrame(X)
    X = reduce_mem_usage(X)
    if name_prefix is not None:
        #Replacing column names by adding prefix to each column name
        X.columns = [ name_prefix + ' ' + c for c in X.columns]
        
    return X

#### Function for Creating Dataset Having Custom Features

In [ ]:
def creating_dataset(data, data_item, data_store_class, items, stores, Date, n_weeks,return_labels=True):
    
    '''Creates dataset having custom features'''


    #Format --> data=(sales,promo)
    sales = data[0]
    promo = data[1]

    #Format --> data_item=(sales_item,promo_item)
    sales_item = data_item[0]
    promo_item = data_item[1]

    #Format --> data_store_class=(store_class_sales,store_class_promo)
    store_class_sales = data_store_class[0]
    store_class_promo = data_store_class[1]
    store_class_index = data_store_class[2]
    del data,data_item,data_store_class
    X = []
    Y = []
    
    #Creating features for every week one by one
    for i in range(n_weeks):

        # Creating features using sales of each store_nbr and item_nbr pair 
        x1 = custom_features(sales, promo, Date)

        # Creating features using sales of each item_nbr 
        x2 = custom_features(sales_item, promo_item, Date, name_prefix='item')
        # Setting item_nbr as index
        x2.index = sales_item.index
        # Re-indexing item_nbr acc. to order of item_nbr in sales dataframe
        x2 = x2.reindex(sales.index.get_level_values(1))    
        # Resetting index with a sequential index and dropping old index
        x2 = x2.reset_index(drop=True)     

        # Creating features using sales of each item_class and store_nbr pair
        x3 = custom_features(store_class_sales, store_class_promo, Date,  name_prefix='store_class')
        # Setting item_class and store_nbr as index
        x3.index = store_class_sales.index
        # Re-indexing item_nbr acc. to order of item_nbr in sales dataframe
        x3 = x3.reindex( pd.MultiIndex.from_frame(store_class_index))
        # Resetting index with a sequential index and dropping old index
        x3 = x3.reset_index(drop=True)   

        #Concatenating(horizontally) all the above custom created features and given store and item features.
        x = pd.concat([x1, x2, x3, items.reset_index(), stores.reset_index()], axis=1)
        #Appending data of each week in final dataset 
        X.append(x)
        
        del x,x1,x2,x3

        #True sales for the following 16 days (that is to be predicted)
        try :
            y = sales[pd.date_range(Date, periods=16)].values
            Y.append(y)
            del y
        except:
            pass
            
        # Adding 7 days i.e shifting to next week.
        Date = Date + timedelta(days=7)
        
    X = pd.concat(X, axis=0)

    if return_labels==True:
        Y = np.concatenate(Y, axis=0)
        return X,Y
    else:
        return X

### Create Feature Set

In [ ]:
def create_feature_set(Date):    
    ''' Creates feature set '''

    
    df_train,df_test,stores,items = read_process()

    sales_2017,promo_2017 = data_storenbr_itemnbr(df_train,df_test)
    promo_item_2017,sales_item_2017 = data_itemnbr(promo_2017,sales_2017)
    items,stores,store_class_promo_2017,store_class_index,store_class_sales_2017=data_itemclass_storenbr(items,stores,sales_2017,promo_2017)

    """#### Defining Data"""

    #data of each store_nbr and item_nbr pair
    data=(sales_2017,promo_2017)

    #data of each item_nbr
    data_item=(sales_item_2017,promo_item_2017)

    #data of each item_class and store_nbr pair
    data_store_class=(store_class_sales_2017,store_class_promo_2017,store_class_index)

    #Deleting unnecessary variables
    del promo_2017,sales_item_2017, promo_item_2017, store_class_sales_2017, store_class_promo_2017


    """#### Creating Feature Set"""

    #no. of weeks for which features have to created
    n_weeks = 1

    features = creating_dataset(data, data_item, data_store_class, items, stores, Date, n_weeks, return_labels=False)


    #Deleting unnecessary variables
    del data, data_item,data_store_class

    return features

## Function 1

In [ ]:
def log_transform(arr):
    ''' Transforms array taking log(x+1) for every element (x) '''

    new_arr=[]
    for value in arr:
        if float(value) > 0:
            value=np.log1p(float(value))
        else:
            value=0
        new_arr.append(value)
    return np.array(new_arr)

In [ ]:
def calculate_nwrmsle(true,pred,weight):
    ''' 
    Calculates Normalized Weighted Root Mean Squared Logarithmic Error (nwrmsle)

    true = true labels
    pred =  predicted labels
    weight = weights of datapoints

    returns nwrmsle '''

    temp=(log_transform(true) - log_transform(pred))**2
    temp = temp * weight
    nwrmsle = np.sqrt(temp.sum() / weight.sum() )
    return nwrmsle

In [ ]:
def predict(X_query):
    ''' Predicts sales using the saved models
        given query dataframe'''
    
    # Loading Top 300 Feature Names (got by training random forest)
    import pickle
    with open('/content/drive/My Drive/self_case_study1/300_filtered_features.pkl','rb') as file:
        filtered_features = pickle.load( file)

    # Loading Saved Models (16 LGBM Models)
    print("Loading Models ...")
    import pickle
    with open('/content/drive/My Drive/self_case_study1/final_models.pkl', 'rb') as file:
        models = pickle.load(file)


    print("Creating Features ...")
    Date = X_query.iloc[0]['date']
    model_no = 0

    if Date > pd.to_datetime('2017-08-16'):
        model_no = (Date  -  pd.to_datetime('2017-08-16')).days
        Date = pd.to_datetime('2017-08-16')

    features = create_feature_set(Date)
    features = features.set_index(["store_nbr", "item_nbr"])

    #Fetching all unique dates
    Dates = X_query.date.unique()
    date_column = [ ]

    y_pred = np.array([])
    temp_df = pd.DataFrame()

    print("Predicting ...")
    for i,Date in tqdm(enumerate(Dates)):

        #Fetching data for each Date
        batch = X_query[X_query['date'] == Date]
        batch = batch.set_index(["store_nbr", "item_nbr"])
        batch = features.reindex(batch.index)
        batch.dropna(axis = 0 , how='all' , inplace=True)
        batch.reset_index(inplace=True)
        if i == 0:
            temp_df = batch[['store_nbr','item_nbr']]
        else:
            temp_df = pd.concat([temp_df,batch[['store_nbr','item_nbr']]])

        #Filtering Features
        X = batch[filtered_features[model_no]]

        #Predicting 
        y_pred = np.append( y_pred , models[model_no].predict(X) )
        model_no = model_no + 1

        for j in range(0,batch.shape[0]):
            date_column.append(Date)

        del X,batch

    #Adding date column
    temp_df['date'] = date_column

    # setting index as store_nbr,item_nbr,date
    temp_df  = temp_df.set_index(["store_nbr", "item_nbr",'date']) 
    X_query = X_query.set_index(["store_nbr", "item_nbr",'date'])

    # Creating Dataframe with test predicitons and setting index same as temp_df Dataframe (i.e. str_nbr, item_nbr)
    df_preds = pd.DataFrame(y_pred , columns = ['unit_sales'] , index=temp_df.index )   

    # Setting names of the indices
    df_preds.index.set_names(["store_nbr", "item_nbr","date"], inplace=True)

    # Joining the given query dataset(X_query) for which predictions were to be made
    # and the dataframe in which predicited values are present
    predicitons = X_query[['id']].join(df_preds, how="left").fillna(0) #The prediciton is only done for 16 days so the other days sales will be filled with 0

    # Converting predicted unit_sales back to orginal form by taking exp(unit_sales) - 1 as it was previously converted using log(unit_sales) + 1
    predicitons["unit_sales"] = np.clip(np.expm1(predicitons["unit_sales"]), 0, 1000)

    return predicitons["unit_sales"].values

In [ ]:
def Function_1(X_query):
    ''' Returns sale predictions given raw input '''

    if type(X_query)== np.ndarray:
        print("Got Array as an Input ...")

        #Re-shaping if the array has only 1 row
        try:
            X_query=X_query.reshape(1,5)
        except:
            pass
        #Creating Dataframe
        X_query=pd.DataFrame(X_query)
        
    elif type(X_query)==pd.DataFrame:
        print("Got Dataframe as an Input ...")

    #Assigning column names
    X_query.columns = ['id','date','store_nbr','item_nbr','onpromotion']
    #Changing datatypes of 'date' and 'onpromotion' columns
    X_query['date']= X_query['date'].astype('datetime64[ns]')
    X_query['onpromotion'] = X_query['onpromotion'].astype('int')
    #Predicting Sales
    y_pred = predict(X_query)

    return X_query,y_pred     

In [ ]:
# Sample Data for Testing Function 1

X_query = pd.read_csv('test.csv')
X_query = X_query.to_numpy()
X_query

array([[125497040, '2017-08-16', 1, 96995, False],
       [125497041, '2017-08-16', 1, 99197, False],
       [125497042, '2017-08-16', 1, 103501, False],
       ...,
       [128867501, '2017-08-31', 54, 2132945, False],
       [128867502, '2017-08-31', 54, 2132957, False],
       [128867503, '2017-08-31', 54, 2134244, False]], dtype=object)

In [ ]:
_,y_pred = Function_1(X_query)

Got Array as an Input ...
Loading Models ...
Creating Features ...
Predicting ...


In [ ]:
y_pred

array([0.21727337, 0.26708205, 0.        , ..., 0.        , 0.        ,
       0.        ])

## Function 2

In [ ]:
def Function_2(X_query,true):
    ''' Returns NWRMSLE score given raw input '''
    
    #Prediciting sales 
    X_query , y_pred = Function_1(X_query)

    #Fetching weights
    items=pd.read_csv("items.csv")
    weight=(X_query.merge( items, on ='item_nbr', how='left')['perishable']) * 0.25 + 1

    #Calculating score
    score = calculate_nwrmsle(true,y_pred,weight.values)
    return score

In [ ]:
# Sample Data for Testing Function 2

X_query = pd.read_csv('train.csv', parse_dates=["date"], skiprows=range(1,104688780))
X_query = X_query[(X_query.date >= pd.to_datetime('2017-07-31')) & (X_query.date < pd.to_datetime('2017-08-16')) ]
X_query['date']=X_query['date'].astype(str)
true = X_query['unit_sales'].values
del X_query['unit_sales']
X_query = X_query.to_numpy()
X_query 

array([[123819696, '2017-07-31', 1, 96995, False],
       [123819697, '2017-07-31', 1, 103520, False],
       [123819698, '2017-07-31', 1, 103665, False],
       ...,
       [125497037, '2017-08-15', 54, 2110456, False],
       [125497038, '2017-08-15', 54, 2113914, True],
       [125497039, '2017-08-15', 54, 2116416, False]], dtype=object)

In [ ]:
score = Function_2(X_query,true)
print("Nwrmsle Score --> ",score)

Got Array as an Input ...
Loading Models ...
Creating Features ...
Predicting ...



Nwrmsle Score -->  0.5897288589434172
